In [ ]:
import json
import os
import os.path
import matplotlib.pyplot as plt
import numpy as np
import glob
import copy
import shutil
import os.path

from copy import deepcopy
from os.path import exists, join
from collections import defaultdict

In [ ]:
def load_algo_names():
    with open('stl_algo_names.txt') as f:
        content = f.readlines()
    content = [line for line in content if not line.startswith('//')]
    content = [line.replace(',', '') for line in content]
    content = [x.replace('"', '') for line in content for x in line.split()]
    # flatten list

    return content

def load_algo_names_by_kind():
    with open('stl_algo_names_by_kind.json') as f:
        content = json.load(f)

    return content

FEATURES_FOLDER = '/home/cdragancea/FBACode/3940_analyze'


In [ ]:
# load the outputs from the analyze/ folder
stats = dict()
analyze_summary = dict()

with open(f"{FEATURES_FOLDER}/analyze_summary.json", "r") as fin:
    analyze_summary = json.load(fin)

counter = 0
for project_name in os.listdir(FEATURES_FOLDER):
    # print(f"Loading overall_stats for {project_name}")
    # if project_name == "analyze_summary.json":
    if project_name == "analyze_summary.json" or project_name == "analyze_summary2.json":
        continue
    if not exists(join(FEATURES_FOLDER, project_name, 'overall_stats')):
        print(f"Skipping {project_name} because it does not have overall_stats. Status in analyze_summary: {analyze_summary[project_name]['analysis']}")
        counter += 1
        # analyze_summary.pop(project_name)
        # shutil.rmtree(join(FEATURES_FOLDER, project_name), ignore_errors=True)
        continue
    with open(join(FEATURES_FOLDER, project_name, 'overall_stats')) as f:
        content = f.read()
    try:
        overall_stats = json.loads(content)
    except json.JSONDecodeError as e:
        str_err = str(e)
        line = int(str_err.split()[3]) - 1
    
    individual_features = json.loads('\n'.join(content.split('\n')[:line]))
    overall_features = json.loads('\n'.join(content.split('\n')[line:]))
    stats[project_name] = {
        'individual_features': individual_features,
        'overall_features': overall_features
    }

project_names = list(stats.keys())
# with open(f"{FEATURES_FOLDER}/analyze_summary2.json", "w") as fout:
#     fout.write(json.dumps(analyze_summary, indent=4))

In [ ]:
print(len(stats.keys()))
# print(stats['abseil']['overall_features']['algorithm type prevalence'])
# print(stats['dolphin-emu']['overall_features']['algorithm type prevalence'])
# print({pname: p for pname, p in stats.items() if 'algorithm type prevalence' in p['overall_features'] and 'make_heap' in p['overall_features']['algorithm type prevalence']})
# for pname, p in stats.items():
#     if p.get("overall_features", None) is None:
#         continue
#     if p['overall_features'].get('algorithm type prevalence', None) is None:
#         continue
#     if p['overall_features']['algorithm type prevalence'].get("make_heap", None) is None:
#         continue
#     print(pname)
#     print(p['overall_features']['algorithm type prevalence']['make_heap'])


In [ ]:
nr_of_algos = []
algo_names = load_algo_names()
# nr_of_algo_uses = {name: 0 for name in algo_names}
nr_of_algo_uses = defaultdict(int)

for name in algo_names:
    nr_of_algo_uses[name] = 0

for project_name in project_names:
    project = stats[project_name]
    if project['overall_features'] is None:
        # print(f"{project_name} has empty overall_features")
        nr_of_algos.append(0)
        continue

    if 'algorithm type prevalence' not in project['overall_features']:
        # print(f"{project_name} has no algorithm type prevalence")
        nr_of_algos.append(0)
        continue
    # print(f'project name: {project_name}')
    if project['overall_features']['algorithm type prevalence'] is None:
        # print(f"{project_name} has empty algorithm type prevalence")
        nr_of_algos.append(0)
        continue
    nr_of_algos.append(sum([v for (_, v) in project['overall_features']['algorithm type prevalence'].items()]))
    for algo_name, algo_uses in project['overall_features']['algorithm type prevalence'].items():
        nr_of_algo_uses[algo_name] += algo_uses

In [ ]:
# number of algos used per project
# unique_values, frequencies = np.unique(nr_of_algos, return_counts=True)
nr_of_algos_per_project = sorted(nr_of_algos, reverse=True)
first_0 = nr_of_algos_per_project.index(0)
first_10 = nr_of_algos_per_project.index(10)

# Step 4: Plot the frequencies
plt.plot(nr_of_algos_per_project)

plt.axvline(x = first_0, color='r', linestyle='--')
plt.text(first_0 + 10, 100, '0 calls', color='black', fontsize=8, ha='left')

plt.axvline(x = first_10, color='r', linestyle='--')
plt.text(first_10 + 10, 100, '10 calls', color='black', fontsize=8, ha='left')

# plt.bar(nr_of_algos_per_project, align='center', alpha=0.5)
plt.xlabel('Projects')
plt.ylabel('Number of algorithm calls in said project')
plt.title('Plot of number of algorithm calls per project, in decreasing order')
plt.xticks([])
plt.grid(True)
plt.show()


In [ ]:
print({k : v for k, v in nr_of_algo_uses.items() if v > 10})
print(sorted(nr_of_algo_uses.items(), key=lambda x: x[1], reverse=True))

In [ ]:
# # frequency of number of algos used
# unique_values, frequencies = np.unique(list(nr_of_algo_uses.values()), return_counts=True)
# print(list(zip(unique_values, frequencies)))

# # Step 4: Plot the frequencies
# plt.bar(unique_values, frequencies, align='center', alpha=0.5)
# plt.xlabel('Number of calls')
# plt.ylabel('Number of algorithms with said number of calls')
# plt.title('Frequency of number of calls to algorithms in projects')
# plt.xticks(unique_values)
# plt.grid(True)
# plt.show()


In [ ]:
# # how often each algo appears
# unique_values, frequencies = np.unique(list(nr_of_algo_uses.values()), return_counts=True)
# print(list(zip(unique_values, frequencies)))

# # Step 4: Plot the frequencies
# plt.bar(unique_values, frequencies, align='center', alpha=0.5)
# plt.xlabel('Number of calls')
# plt.ylabel('Number of algorithms with said number of calls')
# plt.title('Frequency of number of calls to algorithms in projects')
# plt.xticks(unique_values)
# plt.grid(True)
# plt.show()

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Create a wordcloud object
wordcloud = WordCloud(width = 1200, height = 1200, background_color='white')
# wordcloud = WordCloud(width = 1200, height = 1200)

# Generate the wordcloud from a text
text = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua."
wordcloud.generate_from_frequencies(nr_of_algo_uses)

# Display the wordcloud
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

wordcloud.to_file('wordcloud.png')


In [ ]:
# Get SLOC for each project
file_list = [
    "debian-success-test-unified.json",
]

sloc_info = {}

for file_name in file_list:
    with open(f"../examples/{file_name}", "r") as fin:
        data = json.load(fin)["debian"]
        for project in data:
            cpp_lines = [x[1] for x in data[project]["codebase_data"]["sloc"] if x[0] == "cpp"]
            sloc_info[project] = cpp_lines[0]

print(json.dumps(sloc_info, indent=4))

In [ ]:
with open("nr_for_each_algo.txt", "w") as fout:
    for k, v in sorted(list(nr_of_algo_uses.items()), key=lambda x: x[1], reverse=True):
        fout.write(f"{k} {v}\n")

In [ ]:
with open("nr_for_each_proj.txt", "w") as fout:
    lines = []
    for k, v in sorted(list(zip(project_names, nr_of_algos)), key=lambda x: x[1], reverse=True):
        if k in sloc_info:
            lines.append((k, v, sloc_info[k], v / sloc_info[k]))
        # fout.write(f"{k} {v} {sloc_info[k]} {v / sloc_info[k]}\n")
        else:
            lines.append((k, v, 0, 0))
            # fout.write(f"{k} {v}\n")
    
    print(lines[:10])
    for line in sorted(lines, key = lambda x: x[1], reverse=True):
        fout.write(f"{line[0]} {line[1]} {line[2]} {line[3]}\n")
        # fout.write(f"{line[0]} {line[1]}\n")

In [ ]:
def list_algo_uses_for_projects(projects):
    for project_name in projects:
        with open(f"./data/algo_uses_per_project/{project_name}.txt", "w") as fout:
            algo_calls = [(k, v) for k, v in stats[project_name]['overall_features']['algorithm type prevalence'].items() if v > 0]
            algo_calls = sorted(algo_calls, key=lambda x: x[1], reverse=True)

            fout.write(f"Distinct algo calls (> 0) in project: {len(algo_calls)}\n")
            fout.write("algo_name,total_algo_calls\n")
            for algo_name, algo_uses in algo_calls:
                fout.write(f"{algo_name.replace('std::','')},{algo_uses}\n")
            

list_algo_uses_for_projects(["bagel", "opencv", "praat", "gdal", "telegram-desktop"])

In [ ]:
# with open("nr_for_each_algo_no_praat.txt", "w") as fout:
#     this_data = {k: v for k, v in nr_of_algo_uses.items()}
#     # for k, v in stats["praat"]["overall_features"]["algorithm type prevalence"].items():
#     #     this_data[k] -= v
#     for k, v in sorted(list(this_data.items()), key=lambda x: x[1], reverse=True):
#         fout.write(f"{k} {v}\n")

In [ ]:
# Total number of projects
print(f"Total number of projects: {len(project_names)}")

# How many have less than X algos
X = 0
print(f"Projects with less than {X} algo usages: {len([x for x in nr_of_algos if x <= X])}")

In [ ]:
def plot_by_kind(algo_kind, algo_names):
    print(f"Plotting uses for {algo_kind}")
    values = [(name, nr_of_algo_uses[name]) for name in algo_names]
    values = sorted(values, key=lambda x: x[1], reverse=True)

    print(f"Nr of algos that are not used for {algo_kind}: {len([x for x in values if x[1] == 0])}")
    
    zero_values = [x for x in values if x[1] == 0]
    if len(values) > 15:
        values = [x for x in values if x[1] > 0]
    # values.append( ('\n'.join([x[0] for x in zero_values]), 0) )

    values = [(x[0].replace('std::', ''), x[1]) for x in values]

    x_axis = [x[0] for x in values]
    y_axis = [x[1] for x in values]
    # Plot the usage
    # plt.figure(figsize=(7, 9))
    plt.bar(x_axis, y_axis, align='center', alpha=0.5)
    plt.xlabel('Algorithm name')
    plt.ylabel('Number of algorithm calls')
    plt.title(f'Frequency of \"{algo_kind}\" STL algorithms')
    plt.xticks(x_axis, rotation=90 if len(x_axis) > 6 else 0)
    plt.grid(True)

    for i, value in enumerate(values):
        plt.text(i, value[1], str(value[1]), ha='center')
    
    # plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)

    plt.savefig(f"./plots/{algo_kind.replace('/', ' or ')}.png", bbox_inches='tight')
    plt.show()

In [ ]:
algo_names_by_kind = load_algo_names_by_kind()
for algo_kind, algo_names in algo_names_by_kind.items():
    plot_by_kind(algo_kind, algo_names)

In [ ]:
def split_by_char(input, split_char):
    # Need to split while making sure that we are at the same level of < brackets
    assert len(split_char) == 1, "split_char must be a single character"
    level = 0
    current = ""
    result = []
    for c in input:
        if c == "<":
            level += 1
        elif c == ">":
            level -= 1
        if c == split_char and level == 0:
            result.append(current)
            current = ""
        else:
            current += c
    result.append(current)
    result = [x.strip() for x in result]
    return result

def parse_basic_string(type_str):
    # looks like std::basic_string<char, std::char_traits<char>, std::allocator<char> >
    # so basically std::basic_stirng<type, traits, allocator>

    if type_str == "std::basic_string<char, std::char_traits<char>, std::allocator<char> >":
        return "string"
    if type_str == "std::basic_string<wchar_t, std::char_traits<wchar_t>, std::allocator<wchar_t> >":
        return "wstring"
    if type_str == "std::basic_string<char8_t, std::char_traits<char8_t>, std::allocator<char8_t> >":
        return "u8string"
    if type_str == "std::basic_string<char16_t, std::char_traits<char16_t>, std::allocator<char16_t> >":
        return "u16string"
    if type_str == "std::basic_string<char32_t, std::char_traits<char32_t>, std::allocator<char32_t> >":
        return "u32string"
    return f"unknown basic_string <{type_str}>"

def parse_normal_iterator(type_str):
    # looks like normal_iterator<type *, container<type>>
    # or normal_iterator<type *, container<type, allocator<type>>>
    
    # trim the beginning and end >
    type_str = type_str[len("__gnu_cxx::__normal_iterator<"):-1]
    # split by ',' and strip whitespace at end
    parts = split_by_char(type_str, ',')

    # first parameter is a pointer to the type
    container_type = parts[1]

    return f"Iterator on {parse_type(container_type)}"

def parse_back_insert_iterator(type_str):
    # looks like std::back_insert_iterator<>
    # trim the beginning and end >
    type_str = type_str[len("std::back_insert_iterator<"):-1]
    parts = split_by_char(type_str, ',')
    return f"back_insert_iterator<{parse_type(parts[0])}>"

def parse_vector_type(type_str):
    # looks like std::vector <T, std::allocator<T>>
    # trim the beginning and end>
    type_str = type_str[len("std::vector<"):-1]
    parts = split_by_char(type_str, ',')

    return f"vector<{parse_type(parts[0])}>" 

def parse_type(type_str):
    if type_str.endswith("&"):
        return f"{parse_type(type_str[:-1].strip())}&"
    if type_str.startswith("__gnu_cxx::__normal_iterator<"):
        return parse_normal_iterator(type_str)
    if type_str.startswith("std::basic_string<"):
        return parse_basic_string(type_str)
    if type_str.startswith("std::vector"):
        return parse_vector_type(type_str)
    if type_str.startswith("std::back_insert_iterator<"):
        return parse_back_insert_iterator(type_str)

    # check if there is a layer of <>
    if type_str.count('<') < 1 or type_str.count('>') < 1:
        return type_str
    
    outer_type_str_l = type_str[:type_str.index('<') + 1]
    outer_type_str_r = type_str[type_str.rindex('>'):]
    inner_type_str = parse_type(type_str[type_str.index('<')+1 : type_str.rindex('>')])

    if outer_type_str_l.startswith("std::"):
        outer_type_str_l = outer_type_str_l[len("std::"):]

    return outer_type_str_l + inner_type_str + outer_type_str_r

long_ass_type = "__gnu_cxx::__normal_iterator<std::basic_string<char, std::char_traits<char>, std::allocator<char> > *, std::vector<std::basic_string<char, std::char_traits<char>, std::allocator<char> >, std::allocator<std::basic_string<char, std::char_traits<char>, std::allocator<char> > > > >"
print(parse_type(long_ass_type))

# Looking at the parameter type of the most popular algorithms

In [ ]:
def investigate_parameter_instantiation_type(algo_names):
    print(f"Investigating parameter instantiation type for {algo_names}")

    instantiation_types = dict()
    for algo_name in algo_names:
        instantiation_types[algo_name] = defaultdict(int)


    # Go through every *.ast.json in the analyze folder
    files = glob.glob(f"{FEATURES_FOLDER}/**/*.ast.json", recursive = True)
    for file in files:
        # if "telegram" in file:
            # print(f"Skipping the telegram-desktop project: {file}")
            # continue
        # print(f"File: {file}")
        try:
            with open(file, "r") as fin:
                data = json.load(fin)
        except Exception as e:
            print(f"Exception: {e}")
            print(f"Skipping file: {file}")
            continue
        
        if "ala" not in data or data["ala"] is None:
            continue

        for algo_name in algo_names:
            if data["ala"].get("implicit func insts", None) is not None and algo_name in data["ala"]["implicit func insts"]:
                for appearence in data["ala"]["implicit func insts"][algo_name]:
                    # if len(appearence["arguments"]["non-type"]) == 0 or len(appearence["arguments"]["template"]) == 0:
                    #     continue
                    # if len(appearence["arguments"]["type"]) == 1:
                    #     continue
                    # instantiation_types[appearence["arguments"]["type"][0]] += 1
                    # instantiation_types[json.dumps(appearence["arguments"])] += 1
                    types_temp = deepcopy(appearence["arguments"]["type"])
                    types_temp = [parse_type(x) for x in types_temp]
                    instantiation_types[algo_name][json.dumps(types_temp)] += 1
            
            if data["ala"].get("explicit func insts", None) is not None and algo_name in data["ala"]["explicit func insts"]:
                for appearence in data["ala"]["explicit func insts"][algo_name]:
                    # if len(appearence["arguments"]["non-type"]) == 0 or len(appearence["arguments"]["template"]) == 0:
                    #     continue
                    # if len(appearence["arguments"]["type"]) == 1:
                    #     continue
                    # instantiation_types[appearence["arguments"]["type"][0]] += 1
                    # instantiation_types[json.dumps(appearence["arguments"])] += 1
                    types_temp = deepcopy(appearence["arguments"]["type"])
                    types_temp = [parse_type(x) for x in types_temp]
                    instantiation_types[algo_name][json.dumps(types_temp)] += 1
    
    for algo_name in algo_names:
        data = list(instantiation_types[algo_name].items())
        data = sorted(data, key = lambda x: x[1], reverse=True)
        # print(f"Number of different instantiation types: {len(instantiation_types)}")
        # for k,v in instantiation_types:
        #     print(f"{k} ::: {v}")
        
        with open(f"./data/algo_parameter_type/{algo_name}.txt", "w") as fout:
            fout.write(f"Number of different instantiation types: {len(data)}\n")
            fout.write(f"Total number of apparitions: {sum([x[1] for x in data])}\n")
            for k, v in data:
                fout.write(f"{v}  {k}\n")


In [ ]:
investigate_parameter_instantiation_type(["std::move", "std::max", "std::min", "std::fill_n", "std::swap", "std::sort", "std::find", "std::copy", "std::find_if", "std::copy_n"])
# investigate_parameter_instantiation_type(["std::move"])

# investigate_parameter_instantiation_type("std::max")
# investigate_parameter_instantiation_type("std::min")
# investigate_parameter_instantiation_type("std::fill_n")
# investigate_parameter_instantiation_type("std::sort")
# investigate_parameter_instantiation_type("std::find")
# investigate_parameter_instantiation_type("std::copy")
# investigate_parameter_instantiation_type("std::find_if") # really diverse uses
# investigate_parameter_instantiation_type("std::transform") # really diverse uses
# investigate_parameter_instantiation_type("std::for_each")

# For each algorithm, plot how many distinct projects it appears in, grouped by kind

In [ ]:
def plot_distinct_projects_by_kind(algo_kind, algo_names):
    print(f"Plotting uses for {algo_kind}")
    
    values = []
    for algo_name in algo_names:
        projects = [pname for pname, p in stats.items() if p.get("overall_features", None) is not None and p['overall_features'].get('algorithm type prevalence', None) is not None and p['overall_features']['algorithm type prevalence'].get(algo_name, 0) > 0]
        values.append((algo_name, len(projects)))


    # values = [(name, nr_of_algo_uses[name]) for name in algo_names]
    values = sorted(values, key=lambda x: x[1], reverse=True)

    print(f"Nr of algos that are not used for {algo_kind}: {len([x for x in values if x[1] == 0])}")
    
    zero_values = [x for x in values if x[1] == 0]
    if len(values) > 15:
        values = [x for x in values if x[1] > 0]
    # values.append( ('\n'.join([x[0] for x in zero_values]), 0) )

    values = [(x[0].replace('std::', ''), x[1]) for x in values]

    x_axis = [x[0] for x in values]
    y_axis = [x[1] for x in values]
    # Plot the usage
    # plt.figure(figsize=(7, 9))
    plt.bar(x_axis, y_axis, align='center', alpha=0.5)
    plt.xlabel('Algorithm name')
    plt.ylabel('Number of distinct projects it appears in')
    plt.title(f'Frequency of \"{algo_kind}\" STL algorithms')
    plt.xticks(x_axis, rotation=90 if len(x_axis) > 6 else 0)
    plt.grid(True)

    for i, value in enumerate(values):
        plt.text(i, value[1], str(value[1]), ha='center')
    
    # plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)

    plt.savefig(f"./plots/{algo_kind.replace('/', ' or ')} in distinct projects.png", bbox_inches='tight')
    plt.show()



In [ ]:
algo_names_by_kind = load_algo_names_by_kind()
for algo_kind, algo_names in algo_names_by_kind.items():
    plot_distinct_projects_by_kind(algo_kind, algo_names)

In [ ]:
def box_plot_algo_usage(algo_name):
    # For each algorithm, count the number of times it appears in each project
    # Ignore the 0s
    # Box plot of when it is used, HOW MUCH is it actually used?

    print(f"Box plot for {algo_name}")

    values = []
    for pname, p in stats.items():
        if p.get("overall_features", None) is None:
            continue
        if p['overall_features'].get('algorithm type prevalence', None) is None:
            continue
        if p['overall_features']['algorithm type prevalence'].get(algo_name, None) is None:
            continue
        if p['overall_features']['algorithm type prevalence'][algo_name] == 0:
            continue
        values.append(p['overall_features']['algorithm type prevalence'][algo_name])
        
    plt.boxplot(values, patch_artist = True, showfliers = False)

    plt.title(f"Box plot for {algo_name} without outliers")
    plt.grid(True)
    plt.xticks([])
    plt.show()

    plt.boxplot(values, patch_artist = True, showfliers = True)

    plt.title(f"Box plot for {algo_name} with outliers")
    plt.grid(True)
    plt.xticks([])
    plt.show()

def box_plot_algo_usage_by_kind(algo_kind):
    pass

In [ ]:
# box_plot_algo_usage(["std::max", "std::min", "std::sort"])
box_plot_algo_usage("std::max")
box_plot_algo_usage("std::min")
box_plot_algo_usage("std::swap")
box_plot_algo_usage("std::sort")
box_plot_algo_usage("std::find")
box_plot_algo_usage("std::copy")
box_plot_algo_usage("std::find_if")
box_plot_algo_usage("std::transform")
box_plot_algo_usage("std::for_each")
box_plot_algo_usage("std::move")

In [ ]:


# projects_with_no_asts = dict()

# # go over all directories in analyze folder
# for project_name in os.listdir("/home/cdragancea/FBACode/analyze"):
#     if not os.path.isdir(join("/home/cdragancea/FBACode/analyze", project_name)):
#         continue

#     files_in_folder = os.listdir(join("/home/cdragancea/FBACode/analyze", project_name))
#     files_in_folder = [file for file in files_in_folder if file.endswith(".ast.json")]

#     if len(files_in_folder) == 0:
#         projects_with_no_asts[project_name] = 1

# print(len(projects_with_no_asts.keys()))

# with open("/home/cdragancea/FBACode/cache_no_asts.json", "w") as fout:
#     fout.write(json.dumps(projects_with_no_asts, indent = 2))

In [ ]:
# lol = "null\nnull\n"

# with open("/home/cdragancea/FBACode/analyze/zutty/overall_stats", "r") as fin:
#     content = fin.read()

# print(content)
# print(lol == content)

In [ ]:
# # go over all directories in analyze folder
# for project_name in os.listdir("/home/cdragancea/FBACode/analyze"):
#     if project_name not in projects_with_no_asts:
#         continue

#     if not exists(join("/home/cdragancea/FBACode/analyze", project_name, "overall_stats")):
#         projects_with_no_asts.pop(project_name)
#         continue

#     with open(join("/home/cdragancea/FBACode/analyze", project_name, "overall_stats"), "r") as fin:
#         content = fin.read()
    
#     if content != "null\nnull\n":
#         print(f"{project_name} doesn't have null null")


In [ ]:

# with open("/home/cdragancea/FBACode/cache_no_asts.json", "w") as fout:
#     fout.write(json.dumps(projects_with_no_asts, indent = 2))

In [ ]:
smart_ptr_usage = defaultdict(int)
smart_ptr_usage_distinct = defaultdict(int)
# nr_of_pointers_uses = [0 for pname in project_names]
nr_of_pointers_uses = []
SMART_PTR_NAMES = ["std::shared_ptr", "std::weak_ptr", "std::unique_ptr"]

with open("./data/smart_pointers/smart-pointers-per-project.txt", "w") as fout:
    fout.write("project_name,shared_ptr,weak_ptr,unique_ptr\n")
    smart_ptr_per_project = []
    for project_name in project_names:
        project = stats[project_name]
        if project['overall_features'] is None:
            # print(f"{project_name} has empty overall_features")
            nr_of_pointers_uses.append(0)
            smart_ptr_per_project.append((project_name, 0, 0, 0))
            continue

        if 'utility type prevalence' not in project['overall_features']:
            # print(f"{project_name} has no utility type prevalence")
            nr_of_pointers_uses.append(0)
            smart_ptr_per_project.append((project_name, 0, 0, 0))
            continue
        # print(f'project name: {project_name}')
        if project['overall_features']['utility type prevalence'] is None:
            # print(f"{project_name} has empty utility type prevalence")
            nr_of_pointers_uses.append(0)
            smart_ptr_per_project.append((project_name, 0, 0, 0))
            continue
        nr_of_pointers_uses.append(sum([v for (_, v) in project['overall_features']['utility type prevalence'].items()]))
        for smart_ptr_name, smart_ptr_uses in project['overall_features']['utility type prevalence'].items():
            if smart_ptr_name in SMART_PTR_NAMES:
                smart_ptr_usage[smart_ptr_name] += smart_ptr_uses
        smart_ptr_per_project.append((project_name, project['overall_features']['utility type prevalence'].get("std::shared_ptr", 0), project['overall_features']['utility type prevalence'].get("std::weak_ptr", 0), project['overall_features']['utility type prevalence'].get("std::unique_ptr", 0)))

    smart_ptr_per_project.sort(key=lambda x: x[1] + x[2] + x[3], reverse=True)
    for project_name, shared_ptr, weak_ptr, unique_ptr in smart_ptr_per_project:
        fout.write(f"{project_name},{shared_ptr},{weak_ptr},{unique_ptr}\n")

In [ ]:
print(smart_ptr_usage)

In [ ]:
def plot_smart_ptr(smart_ptr_names):
    print(f"Plotting uses for smart pointers")
    
    values = []
    for smart_ptr in smart_ptr_names:
        projects = [p['overall_features']['utility type prevalence'][smart_ptr] for pname, p in stats.items() if p.get("overall_features", None) is not None and \
                    p['overall_features'].get('utility type prevalence', None) is not None and p['overall_features']['utility type prevalence'].get(smart_ptr, 0) > 0]
        values.append((smart_ptr, sum(projects)))


    # values = [(name, nr_of_algo_uses[name]) for name in algo_names]
    values = sorted(values, key=lambda x: x[1], reverse=True)

    values = [(x[0].replace('std::', ''), x[1]) for x in values]

    x_axis = [x[0] for x in values]
    y_axis = [x[1] for x in values]
    # Plot the usage
    # plt.figure(figsize=(7, 9))
    plt.bar(x_axis, y_axis, align='center', alpha=0.5)
    plt.xlabel('Smart pointer name')
    plt.ylabel('Number of times it appears in projects')
    plt.title(f'Frequency of smart pointer usage')
    plt.xticks(x_axis, rotation=90 if len(x_axis) > 6 else 0)
    plt.grid(True)

    for i, value in enumerate(values):
        plt.text(i, value[1], str(value[1]), ha='center')
    
    # plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)

    plt.savefig(f"./plots/smart pointers in projects.png", bbox_inches='tight')
    plt.show()

def plot_smart_ptr_distinct(smart_ptr_names):
    print(f"Plotting uses for smart pointers")
    
    values = []
    for smart_ptr in smart_ptr_names:
        projects = [p['overall_features']['utility type prevalence'][smart_ptr] for pname, p in stats.items() if p.get("overall_features", None) is not None and \
                    p['overall_features'].get('utility type prevalence', None) is not None and p['overall_features']['utility type prevalence'].get(smart_ptr, 0) > 0]
        values.append((smart_ptr, len(projects)))


    # values = [(name, nr_of_algo_uses[name]) for name in algo_names]
    values = sorted(values, key=lambda x: x[1], reverse=True)

    values = [(x[0].replace('std::', ''), x[1]) for x in values]

    x_axis = [x[0] for x in values]
    y_axis = [x[1] for x in values]
    # Plot the usage
    # plt.figure(figsize=(7, 9))
    plt.bar(x_axis, y_axis, align='center', alpha=0.5)
    plt.xlabel('Smart pointer name')
    plt.ylabel('Number of distinct projects it appears in')
    plt.title(f'Frequency of smart pointer usage')
    plt.xticks(x_axis, rotation=90 if len(x_axis) > 6 else 0)
    plt.grid(True)

    for i, value in enumerate(values):
        plt.text(i, value[1], str(value[1]), ha='center')
    
    # plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)

    plt.savefig(f"./plots/smart pointers in distinct projects.png", bbox_inches='tight')
    plt.show()

plot_smart_ptr(SMART_PTR_NAMES)
plot_smart_ptr_distinct(SMART_PTR_NAMES)

In [ ]:
def box_plot_smart_ptr_usage(smart_ptr_name):
    # For each smart pointer, count the number of times it appears in each project
    # Ignore the 0s
    # Box plot of when it is used, HOW MUCH is it actually used?

    print(f"Box plot for {smart_ptr_name}")

    values = []
    for pname, p in stats.items():
        if p.get("overall_features", None) is None:
            continue
        if p['overall_features'].get('utility type prevalence', None) is None:
            continue
        if p['overall_features']['utility type prevalence'].get(smart_ptr_name, None) is None:
            continue
        if p['overall_features']['utility type prevalence'][smart_ptr_name] == 0:
            continue
        values.append(p['overall_features']['utility type prevalence'][smart_ptr_name])

    values = sorted(values)[:-5]

    plt.boxplot(values, patch_artist = True, showfliers = False)

    plt.title(f"Box plot for {smart_ptr_name} without outliers")
    plt.grid(True)
    plt.xticks([])
    plt.show()

    plt.boxplot(values, patch_artist = True, showfliers = True)

    plt.title(f"Box plot for {smart_ptr_name} with outliers")
    plt.grid(True)
    plt.xticks([])
    plt.show()

In [ ]:
box_plot_smart_ptr_usage("std::shared_ptr")
box_plot_smart_ptr_usage("std::weak_ptr")
box_plot_smart_ptr_usage("std::unique_ptr")

In [ ]:

def get_venn_smart_ptr_usage():
    venn_smart_ptr_usage = []
    for ss in range(1, (1 << len(SMART_PTR_NAMES))):
        print_flag = False

        smart_ptr_names = [SMART_PTR_NAMES[i] for i in range(len(SMART_PTR_NAMES)) if ss & (1 << i) > 0]
        smart_ptr_names_absent = [SMART_PTR_NAMES[i] for i in range(len(SMART_PTR_NAMES)) if ss & (1 << i) == 0]
        if len(smart_ptr_names) == 0:
            continue
        count = 0

        for pname, p in stats.items():
            if p.get("overall_features", None) is None:
                continue
            if p['overall_features'].get('utility type prevalence', None) is None:
                continue
            flag = True
            for smart_ptr_name in smart_ptr_names:
                if p['overall_features']['utility type prevalence'].get(smart_ptr_name, None) is None:
                    flag = False
                    continue
                if p['overall_features']['utility type prevalence'][smart_ptr_name] == 0:
                    flag = False
                    continue
            for smart_ptr_name in smart_ptr_names_absent:
                if p['overall_features']['utility type prevalence'].get(smart_ptr_name, None) is None:
                    continue
                if p['overall_features']['utility type prevalence'][smart_ptr_name] == 0:
                    continue
                flag = False
            if flag:
                if not print_flag:
                    print_flag = True
                    print(f"Project {pname} uses only {smart_ptr_names}")
                count += 1
        venn_smart_ptr_usage.append(count)

    return tuple(venn_smart_ptr_usage)

In [ ]:
from matplotlib import pyplot as plt
from matplotlib_venn import venn2, venn3, venn3_unweighted

# Example data
set1 = set(['A', 'B', 'C', 'D'])
set2 = set(['B', 'C', 'D', 'E'])

# Create Venn diagram
# venn2([set1, set2], ('Set 1', 'Set 2'))
venn3_unweighted(subsets = get_venn_smart_ptr_usage(), set_labels = tuple(SMART_PTR_NAMES))




# Add labels
plt.title('Number of projects that use smart pointers')
# save plot
plt.savefig('./plots/venn_smart_pointers.png')
plt.show()


In [ ]:
def investigate_smart_ptr_parameter_instantiation_type(smart_ptr_names):
    print(f"Investigating parameter instantiation type for {smart_ptr_names}")

    instantiation_types = dict()
    for smart_ptr in smart_ptr_names:
        instantiation_types[smart_ptr] = defaultdict(int)


    # Go through every *.ast.json in the analyze folder
    files = glob.glob(f"{FEATURES_FOLDER}/**/*.ast.json", recursive = True)
    for file in files:
        # if "telegram" in file:
            # print(f"Skipping the telegram-desktop project: {file}")
            # continue
        # print(f"File: {file}")
        try:
            with open(file, "r") as fin:
                data = json.load(fin)
        except Exception as e:
            print(f"Exception: {e}")
            print(f"Skipping file: {file}")
            continue
        
        if data.get("ula", None) is None:
            continue

        for smart_ptr in smart_ptr_names:
            if data["ula"].get("implicit class insts", None) is not None and smart_ptr in data["ula"]["implicit class insts"]:
                for appearence in data["ula"]["implicit class insts"][smart_ptr]:
                    # if len(appearence["arguments"]["non-type"]) == 0 or len(appearence["arguments"]["template"]) == 0:
                    #     continue
                    # if len(appearence["arguments"]["type"]) == 1:
                    #     continue
                    # instantiation_types[appearence["arguments"]["type"][0]] += 1
                    # instantiation_types[json.dumps(appearence["arguments"])] += 1
                    types_temp = deepcopy(appearence["arguments"]["type"])
                    types_temp = [parse_type(x) for x in types_temp]
                    instantiation_types[smart_ptr][json.dumps(types_temp)] += 1
            
            if data["ula"].get("explicit class insts", None) is not None and smart_ptr in data["ula"]["explicit class insts"]:
                for appearence in data["ula"]["explicit class insts"][smart_ptr]:
                    # if len(appearence["arguments"]["non-type"]) == 0 or len(appearence["arguments"]["template"]) == 0:
                    #     continue
                    # if len(appearence["arguments"]["type"]) == 1:
                    #     continue
                    # instantiation_types[appearence["arguments"]["type"][0]] += 1
                    # instantiation_types[json.dumps(appearence["arguments"])] += 1
                    types_temp = deepcopy(appearence["arguments"]["type"])
                    types_temp = [parse_type(x) for x in types_temp]
                    instantiation_types[smart_ptr][json.dumps(types_temp)] += 1
    
    for smart_ptr in smart_ptr_names:
        data = list(instantiation_types[smart_ptr].items())
        data = sorted(data, key = lambda x: x[1], reverse=True)
        # print(f"Number of different instantiation types: {len(instantiation_types)}")
        # for k,v in instantiation_types:
        #     print(f"{k} ::: {v}")
        
        with open(f"./data/smart_ptr_parameter_type/{smart_ptr}.txt", "w") as fout:
            fout.write(f"Number of different instantiation types: {len(data)}\n")
            fout.write(f"Total number of apparitions: {sum([x[1] for x in data])}\n")
            for k, v in data:
                fout.write(f"{v}  {k}\n")


In [ ]:
investigate_smart_ptr_parameter_instantiation_type(SMART_PTR_NAMES)

In [ ]:
lol = [pname for pname, p in stats.items() if p.get("overall_features", None) is not None and p['overall_features'].get('algorithm type prevalence', None) is not None and p['overall_features']['algorithm type prevalence'].get("std::move", 0) > 0]
print(len(lol))

In [ ]:
for_loop_usage = defaultdict(int)
for_loop_usage_distinct = defaultdict(int)
nr_of_loop_uses = []
LOOP_KINDS = ["for", "range-for", "while", "do-while"]

with open("./data/loop_kinds/for-loops-per-project.txt", "w") as fout:
    fout.write("project_name,for,range-for,while,do-while\n")
    loop_kinds_per_project = []
    for project_name in project_names:
        project = stats[project_name]
        if project['overall_features'] is None:
            # print(f"{project_name} has empty overall_features")
            nr_of_loop_uses.append(0)
            loop_kinds_per_project.append((project_name, 0, 0, 0, 0))
            continue

        if 'loop kind prevalences' not in project['overall_features']:
            # print(f"{project_name} has no utility type prevalence")
            nr_of_loop_uses.append(0)
            loop_kinds_per_project.append((project_name, 0, 0, 0, 0))
            continue
        # print(f'project name: {project_name}')
        if project['overall_features']['loop kind prevalences'] is None:
            # print(f"{project_name} has empty utility type prevalence")
            nr_of_loop_uses.append(0)
            loop_kinds_per_project.append((project_name, 0, 0, 0, 0))
            continue
        nr_of_loop_uses.append(sum([v for (_, v) in project['overall_features']['loop kind prevalences'].items()]))
        for loop_name, loop_uses in project['overall_features']['loop kind prevalences'].items():
            if loop_name in LOOP_KINDS:
                for_loop_usage[loop_name] += loop_uses
        
        loop_kinds_per_project.append((project_name, project['overall_features']['loop kind prevalences'].get("for", 0), project['overall_features']['loop kind prevalences'].get("range-for", 0), project['overall_features']['loop kind prevalences'].get("while", 0), project['overall_features']['loop kind prevalences'].get("do-while", 0)))
    
    loop_kinds_per_project.sort(key=lambda x: x[1] + x[2] + x[3] + x[4], reverse=True)
    for project_name, for_loop, range_for, while_loop, do_while in loop_kinds_per_project:
        fout.write(f"{project_name},{for_loop},{range_for},{while_loop},{do_while}\n")

In [ ]:
print(for_loop_usage)

In [ ]:
def plot_for_loops(loop_names):
    print(f"Plotting uses for loop kinds")
    
    values = []
    for loop_name in loop_names:
        projects = [p['overall_features']['loop kind prevalences'][loop_name] for pname, p in stats.items() if p.get("overall_features", None) is not None \
                    and p['overall_features'].get('loop kind prevalences', None) is not None and p['overall_features']['loop kind prevalences'].get(loop_name, 0) > 0]
        values.append((loop_name, sum(projects)))


    # values = [(name, nr_of_algo_uses[name]) for name in algo_names]
    values = sorted(values, key=lambda x: x[1], reverse=True)

    values = [(x[0].replace('std::', ''), x[1]) for x in values]

    x_axis = [x[0] for x in values]
    y_axis = [x[1] for x in values]
    # Plot the usage
    # plt.figure(figsize=(7, 9))
    plt.bar(x_axis, y_axis, align='center', alpha=0.5)
    plt.xlabel('Loop kind name')
    plt.ylabel('Number of times it appears in projects')
    plt.title(f'Frequency of loop kinds')
    plt.xticks(x_axis, rotation=90 if len(x_axis) > 6 else 0)
    plt.grid(True)

    for i, value in enumerate(values):
        plt.text(i, value[1], str(value[1]), ha='center')
    
    # plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)

    plt.savefig(f"./plots/loop kinds in projects.png", bbox_inches='tight')
    plt.show()

def plot_for_loops_distinct(loop_names):
    print(f"Plotting uses for loop kinds")
    
    values = []
    for loop_name in loop_names:
        projects = [p['overall_features']['loop kind prevalences'][loop_name] for pname, p in stats.items() if p.get("overall_features", None) is not None \
                    and p['overall_features'].get('loop kind prevalences', None) is not None and p['overall_features']['loop kind prevalences'].get(loop_name, 0) > 0]
        values.append((loop_name, len(projects)))


    # values = [(name, nr_of_algo_uses[name]) for name in algo_names]
    values = sorted(values, key=lambda x: x[1], reverse=True)

    values = [(x[0].replace('std::', ''), x[1]) for x in values]

    x_axis = [x[0] for x in values]
    y_axis = [x[1] for x in values]
    # Plot the usage
    # plt.figure(figsize=(7, 9))
    plt.bar(x_axis, y_axis, align='center', alpha=0.5)
    plt.xlabel('Loop kind name')
    plt.ylabel('Number of distinct projects it appears in')
    plt.title(f'Frequency of loop kinds')
    plt.xticks(x_axis, rotation=90 if len(x_axis) > 6 else 0)
    plt.grid(True)

    for i, value in enumerate(values):
        plt.text(i, value[1], str(value[1]), ha='center')
    
    # plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)

    plt.savefig(f"./plots/loop kinds in distinct projects.png", bbox_inches='tight')
    plt.show()

plot_for_loops(LOOP_KINDS)
plot_for_loops_distinct(LOOP_KINDS)

In [ ]:
from matplotlib import pyplot as plt
from matplotlib_venn import venn2, venn2_unweighted

def get_venn_for_loop_usage():
    for_vs_range_for = [0, 0, 0, 0]
    print_flag = [False, False, False, False]
    # just simple for loop
    for pname, project in stats.items():
        for_usage = 0
        range_for_usage = 0

        if project.get("overall_features", None) is not None and project['overall_features'].get('loop kind prevalences', None) is not None and project['overall_features']['loop kind prevalences'].get("for", 0) > 0:
            for_usage += 1
        
        if project.get("overall_features", None) is not None and project['overall_features'].get('loop kind prevalences', None) is not None and project['overall_features']['loop kind prevalences'].get("range-for", 0) > 0:
            range_for_usage += 1

        if for_usage > 0 and range_for_usage > 0:
            if not print_flag[3]:
                print_flag[3] = True
                print(f"Project {pname} uses both for and range-for")
            for_vs_range_for[3] += 1
        elif for_usage > 0:
            if not print_flag[1]:
                print_flag[1] = True
                print(f"Project {pname} uses only for")
            for_vs_range_for[2] += 1
        elif range_for_usage > 0:
            if not print_flag[2]:
                print_flag[2] = True
                print(f"Project {pname} uses only range-for")
            for_vs_range_for[1] += 1
        else:
            if not print_flag[0]:
                print_flag[0] = True
                print(f"Project {pname} uses neither for nor range-for")
            for_vs_range_for[0] += 1
    return for_vs_range_for

# Example data
# set1 = set(['A', 'B', 'C', 'D'])
# set2 = set(['B', 'C', 'D', 'E'])

# Create Venn diagram
# venn2([set1, set2], ('Set 1', 'Set 2'))
venn2_unweighted(subsets = get_venn_for_loop_usage(), set_labels = ("for", "range-for"))




# Add labels
plt.title('Number of projects that use for vs range-for')
# save plot
plt.savefig('./plots/for_vs_range_for.png')
plt.show()


In [ ]:
alias_typedef_usage = defaultdict(int)
alias_typedef_usage_distinct = defaultdict(int)
nr_alias_typedef_uses = []
ALIAS_KINDS = ["typedef", "alias", "typedef template", "alias templates"]

with open("./data/alias_kinds/alias-uses-per-project.txt", "w") as fout:
    fout.write("project_name,typedef,alias,typedef template,alias templates\n")
    alias_kinds_per_project = []
    for project_name in project_names:
        project = stats[project_name]
        if project['overall_features'] is None:
            # print(f"{project_name} has empty overall_features")
            nr_alias_typedef_uses.append(0)
            alias_kinds_per_project.append((project_name, 0, 0, 0, 0))
            continue

        if 'prevalence of typedef/using' not in project['overall_features']:
            # print(f"{project_name} has no utility type prevalence")
            nr_alias_typedef_uses.append(0)
            alias_kinds_per_project.append((project_name, 0, 0, 0, 0))
            continue
        # print(f'project name: {project_name}')
        if project['overall_features']['prevalence of typedef/using'] is None:
            # print(f"{project_name} has empty utility type prevalence")
            nr_alias_typedef_uses.append(0)
            alias_kinds_per_project.append((project_name, 0, 0, 0, 0))
            continue
        nr_alias_typedef_uses.append(sum([v for (_, v) in project['overall_features']['prevalence of typedef/using'].items()]))
        for alias_name, alias_uses in project['overall_features']['prevalence of typedef/using'].items():
            if alias_name in ALIAS_KINDS:
                alias_typedef_usage[alias_name] += alias_uses
        
    alias_kinds_per_project.sort(key=lambda x: x[1] + x[2] + x[3] + x[4], reverse=True)
    for project_name, typedef, alias, typedef_template, alias_template in alias_kinds_per_project:
        fout.write(f"{project_name},{typedef},{alias},{typedef_template},{alias_template}\n")


print(alias_typedef_usage)

In [ ]:
def plot_alias_typedef(alias_kinds):
    print(f"Plotting uses for alias kinds")
    
    values = []
    for alias_kind in alias_kinds:
        projects = [p['overall_features']['prevalence of typedef/using'][alias_kind] for pname, p in stats.items() if p.get("overall_features", None) is not None \
                    and p['overall_features'].get('prevalence of typedef/using', None) is not None and p['overall_features']['prevalence of typedef/using'].get(alias_kind, 0) > 0]
        values.append((alias_kind, sum(projects)))


    # values = [(name, nr_of_algo_uses[name]) for name in algo_names]
    values = sorted(values, key=lambda x: x[1], reverse=True)

    values = [(x[0].replace('std::', ''), x[1]) for x in values]

    x_axis = [x[0] for x in values]
    y_axis = [x[1] for x in values]
    # Plot the usage
    # plt.figure(figsize=(7, 9))
    plt.bar(x_axis, y_axis, align='center', alpha=0.5)
    plt.xlabel('Alias kind name')
    plt.ylabel('Number of times it appears in projects')
    plt.title(f'Frequency of alias kinds')
    plt.xticks(x_axis, rotation=90 if len(x_axis) > 6 else 0)
    plt.grid(True)

    for i, value in enumerate(values):
        plt.text(i, value[1], str(value[1]), ha='center')
    
    # plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)

    plt.savefig(f"./plots/alias kinds in projects.png", bbox_inches='tight')
    plt.show()

def plot_alias_typedef_distinct(alias_kinds):
    print(f"Plotting uses for loop kinds")
    
    values = []
    for alias_kind in alias_kinds:
        projects = [p['overall_features']['prevalence of typedef/using'][alias_kind] for pname, p in stats.items() if p.get("overall_features", None) is not None \
                    and p['overall_features'].get('prevalence of typedef/using', None) is not None and p['overall_features']['prevalence of typedef/using'].get(alias_kind, 0) > 0]
        values.append((alias_kind, len(projects)))


    # values = [(name, nr_of_algo_uses[name]) for name in algo_names]
    values = sorted(values, key=lambda x: x[1], reverse=True)

    values = [(x[0].replace('std::', ''), x[1]) for x in values]

    x_axis = [x[0] for x in values]
    y_axis = [x[1] for x in values]
    # Plot the usage
    # plt.figure(figsize=(7, 9))
    plt.bar(x_axis, y_axis, align='center', alpha=0.5)
    plt.xlabel('Loop kind name')
    plt.ylabel('Number of distinct projects it appears in')
    plt.title(f'Frequency of alias kinds')
    plt.xticks(x_axis, rotation=90 if len(x_axis) > 6 else 0)
    plt.grid(True)

    for i, value in enumerate(values):
        plt.text(i, value[1], str(value[1]), ha='center')
    
    # plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)

    plt.savefig(f"./plots/alias kinds in distinct projects.png", bbox_inches='tight')
    plt.show()

plot_alias_typedef(ALIAS_KINDS)
plot_alias_typedef_distinct(ALIAS_KINDS)

In [ ]:
from matplotlib import pyplot as plt
from matplotlib_venn import venn2, venn2_unweighted

def get_venn_alias_kind_usage():
    typedef_vs_using = [0, 0, 0, 0]
    print_flag = [False, False, False, False]

    # just simple for loop
    for pname, project in stats.items():
        typedef_usage = 0
        using_usage = 0

        if project.get("overall_features", None) is not None and project['overall_features'].get('prevalence of typedef/using', None) is not None and project['overall_features']['prevalence of typedef/using'].get("typedef", 0) > 0:
            typedef_usage += 1
        
        if project.get("overall_features", None) is not None and project['overall_features'].get('prevalence of typedef/using', None) is not None and project['overall_features']['prevalence of typedef/using'].get("using", 0) > 0:
            using_usage += 1
        
        if typedef_usage > 0 and using_usage > 0:
            if not print_flag[3]:
                print_flag[3] = True
                print(f"Project {pname} uses both typedef and using")
            typedef_vs_using[3] += 1
        elif typedef_usage > 0:
            if not print_flag[2]:
                print_flag[2] = True
                print(f"Project {pname} uses only typedef")
            typedef_vs_using[2] += 1
        elif using_usage > 0:
            if not print_flag[1]:
                print_flag[1] = True
                print(f"Project {pname} uses only using")
            typedef_vs_using[1] += 1
        else:
            if not print_flag[0]:
                print_flag[0] = True
                print(f"Project {pname} uses neither typedef nor using")
            typedef_vs_using[0] += 1

    return typedef_vs_using

# Create Venn diagram
# venn2([set1, set2], ('Set 1', 'Set 2'))
venn2_unweighted(subsets = get_venn_alias_kind_usage(), set_labels = ("typedef", "using"))




# Add labels
plt.title('Number of projects that use typedef vs using')
# save plot
plt.savefig('./plots/typedef_vs_using.png')
plt.show()
